# Exploratory Data Analysis for High School Running Continued Success Factors 


#### - The data are scraped from several running websites 


# Does the Athlete Make the Coach or the Coach Make the Athlete?
OR
# Does Success in High School Running Predict Success in College?

California high schools have produced hundreds of amazing runners that have gone on to continue to run and compete in college.  Are there any lessons learned or obvious trends in the data that can help predict success factors for star high school runners as they attempt to choose the "right" university for them
* What are the strongest factors of succes
s for high school running?
* Where do California High School Runners go to run in college?
* Can we predict whether a high school athlete will continue be successful at their chosen college level program?

Deciding to compete in college at the NCAA Division 1 level takes a high level of success in high school and focus and determination in college.  Not all high schoolers will go on to continued success.  Is this a matter of their collegiate academic pressures? their changing interests and expanded horizons that college environment affords? Does the level of training or over-training in high school determine if they still have room for development in college?  

What take-aways will the data reveal? Some data sources to investigate: 
** https://www.xcstats.com/search-xc-california.php - various XC and track stats are kept here
** https://www.rtspt.com/events/cif/xc2022/mp/#event1 - CIF XC State Results from past 10 years
** https://www.strava.com/activities/8387865995/overview - Strava contains a wealth of runner information and does have a public API - I also am trying to meet with Strava's head AI Data Scientist
** https://athletic.net  - contains athlete profiles for both High School and Collegiate Records/times 
** https://www.milesplit.com/signings - several places provide data on who signed to go run at which college

Data Collection: will have to be manual and pieced together from various sources
** Pull lists of top California runners (on state course or local Crystal Springs XC course) 
** Determine where runners went to college and if they ran
** Obtain Athlete data - Name, DOB, Multi-Sport?, Years competing in HS?, Years Competing in MS?, Years Conmpeting in College?, Event Specialty, HS Events, College Events, GoPro? (Y/N), 
** Obtain High School data - School Name, Coach, Division/Size, Graduation Year, Location, Private/Public,  HS Event PRs
** Obtain College data - School Name, Coach, Division/Size, Graduation Year, Gap Year?, Location, In/Out of State?, Event PRS

Question: Is there a correlation or trend between where the athlete went to HS and College and whether that athlete will continue to develop/improve in their main (2) events? Do some high schools just create superstar runners? Do some colleges create superstar runners? Are there any factors that make it more likely to have a full HS/Collegiate running career? Prediction for who might go pro?

# Standard Package Imports

In [3]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#matplotlib.style.use("Solarize_Light2")
matplotlib.style.use("ggplot")

%matplotlib inline

# Standard functions utilized throughout the notebook

In [4]:
def check_null(df):
    """
    Returns percentage of rows containing missing data
    """
    return df.isna().sum() * 100/len(df)


def get_missing_dates(series, start_date, end_date, freq="D"):
    """
    Returns the dates which are missing in the series
    date_sr between the start_date and end_date
    
    series: Series consisting of date
    start_date: Start date in String format
    end_date: End date in String format
    """
    return pd.date_range(
        start=start_date, end=end_date, freq=freq).difference(series)


def check_duplicate(df, subset):
    """
    Returns if there are any duplicate rows in the DataFrame.
    
    df: DataFrame under consideration
    subset: Optional List of feature names based on which 
            duplicate rows are being identified. 
    """
    if subset is not None: 
        return df.duplicated(subset=subset, keep=False).sum()
    else:
        return df.duplicated(keep=False).sum()


def create_date_features(source_df, target_df, feature_name):
    '''
    Create new features related to dates
    
    source_df : DataFrame consisting of the timestamp related feature
    target_df : DataFrame where new features will be added
    feature_name : Name of the feature of date type which needs to be decomposed.
    '''
    target_df.loc[:, 'year'] = source_df.loc[:, feature_name].dt.year.astype('uint16')
    target_df.loc[:, 'month'] = source_df.loc[:, feature_name].dt.month.astype('uint8')
    target_df.loc[:, 'quarter'] = source_df.loc[:, feature_name].dt.quarter.astype('uint8')
    target_df.loc[:, 'weekofyear'] = source_df.loc[:, feature_name].dt.isocalendar().week.astype('uint8')
    
    target_df.loc[:, 'hour'] = source_df.loc[:, feature_name].dt.hour.astype('uint8')
    
    target_df.loc[:, 'day'] = source_df.loc[:, feature_name].dt.day.astype('uint8')
    target_df.loc[:, 'dayofweek'] = source_df.loc[:, feature_name].dt.dayofweek.astype('uint8')
    target_df.loc[:, 'dayofyear'] = source_df.loc[:, feature_name].dt.dayofyear.astype('uint8')
    target_df.loc[:, 'is_month_start'] = source_df.loc[:, feature_name].dt.is_month_start
    target_df.loc[:, 'is_month_end'] = source_df.loc[:, feature_name].dt.is_month_end
    target_df.loc[:, 'is_quarter_start']= source_df.loc[:, feature_name].dt.is_quarter_start
    target_df.loc[:, 'is_quarter_end'] = source_df.loc[:, feature_name].dt.is_quarter_end
    target_df.loc[:, 'is_year_start'] = source_df.loc[:, feature_name].dt.is_year_start
    target_df.loc[:, 'is_year_end'] = source_df.loc[:, feature_name].dt.is_year_end
    
    # This is of type object
    target_df.loc[:, 'month_year'] = source_df.loc[:, feature_name].dt.to_period('M')
    
    return target_df


def plot_boxh_groupby(df, feature_name, by):
    """
    Box plot with groupby
    
    df: DataFrame
    feature_name: Name of the feature to be plotted
    by: Name of the feature based on which groups are created
    """
    df.boxplot(column=feature_name, by=by, vert=False, 
                              figsize=(10, 6))
    plt.title(f'Distribution of {feature_name} by {by}')
    plt.show()
    

def plot_hist(df, feature_name, kind='hist', bins=100, log=True):
    """
    Plot histogram.
    
    df: DataFrame
    feature_name: Name of the feature to be plotted.
    """
    if log:
        df[feature_name].apply(np.log1p).plot(kind='hist', 
                                              bins=bins, 
                                              figsize=(15, 5), 
                                              title=f'Distribution of log1p[{feature_name}]')
    else:
        df[feature_name].plot(kind='hist', 
                              bins=bins, 
                              figsize=(15, 5), 
                              title=f'Distribution of {feature_name}')
    plt.show()


def plot_ts(series, figsize=(20, 6), title=None, xlabel="", ylabel=""):
    """
    Plot Time Series data. The series object should have date or time as index.
    
    series: Series object to be plotted.
    """
    series.plot(figsize=figsize, title=title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()


def plot_barh(df, feature_name, normalize=True, 
              kind='barh', figsize=(15,5), sort_index=False, title=None):
    """
    Plot barh for a particular feature
    
    kind : Type of the plot
    
    """
    if sort_index==True:
        df[feature_name].value_counts(
                normalize=normalize, dropna=False).sort_index().plot(
                kind=kind, figsize=figsize, grid=True,
                title=title)
    else:   
        df[feature_name].value_counts(
                normalize=normalize, dropna=False).sort_values().plot(
                kind=kind, figsize=figsize, grid=True,
                title=title)
    
    plt.legend()
    plt.show()


def plot_boxh(df, feature_name, kind='box', log=True):
    """
    Box plot
    """
    if log:
        df[feature_name].apply(np.log1p).plot(kind='box', vert=False, 
                                                  figsize=(10, 6), 
                                                  title=f'Distribution of log1p[{feature_name}]')
    else:
        df[feature_name].plot(kind='box', vert=False, 
                              figsize=(10, 6), 
                              title=f'Distribution of {feature_name}')
    plt.show()
    

def plot_scatter(df, feature_x, feature_y, figsize=(10,10), 
                 title=None, xlabel=None, ylabel=None):
    """
    Plot satter     
    """
    df.plot.scatter(feature_x, feature_y, 
                    figsize=(8, 6), title=title, 
                    legend=None)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

# Data Description

The challenge with this project was the initial data collection - there is no one source of high school and college running data but rather multiple sites depending on the timer/company used for the various races and depending on the geographic location of the high school and then colleges. 

**Geographic Scoping:** the decision was made to focus on California high school runners since the CIF California State XC results are the most all-encompassing and enclusive of the runners in the state broken across five different "divisions" based on school size. 

**Gender Specific:** the decision for this analysis was to focus on the development of male high school runners through to the collegiate and professional level since there are dramatic differences in the timing and pace of female running development/maturity versus male maturity and, due to a number of factors, the belief is that due to the introduction of testosterone in the male development cycle it is a perhaps more predictive progression to follow. In any case, the decision was made to only follow the trajectory of male runners. 

**Year Scoping:** Due to the complications of the Covid-19 pandemic on the athletes' running careers and college plans this project decided to only focus on athletes from the high school years 2014-2016. That would allow the assessment of their progress into collegiate running to be 2016-2020 timeframe before the pandemic. 


There are the following data CSV files compiled manually from various websites: 

   **California State XC Results (CIF_XC.csv):** 
   It consists of the state championship 5K cross country results by athlete, division, 
       school, and district/section, along with the 5K time and average pace run per 
       athlete.  The data includes the top 30 male runners from each division.

**NOTE: currently this file only has results from the year 2016 for this EDA (Exploratory Data Assessment) phase - the additional data will be added by the end of this project but it is time consuming so for this stage just 2016 data is assessed**

In [7]:
# Read the data
DATA_DIR = "/Users/tinalount/Desktop/Data/"

XC_2016_df = pd.read_csv(f"{DATA_DIR}/CIF_XC_2016.csv")

In [8]:
XC_2016_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   YEAR               170 non-null    int64 
 1   DATE               170 non-null    object
 2   MEET               170 non-null    object
 3   COURSE             170 non-null    object
 4   DISTANCE (meters)  170 non-null    int64 
 5   WEATHER            170 non-null    object
 6   WEATHER2           170 non-null    int64 
 7   DIVISION           170 non-null    int64 
 8   PLACE              170 non-null    int64 
 9   FIRST_NAME         170 non-null    object
 10  LAST_NAME          170 non-null    object
 11  FULL_NAME          170 non-null    object
 12  YEAR.1             170 non-null    object
 13  SCHOOL             170 non-null    object
 14  SECTION            170 non-null    object
 15  OVERALL TIME       170 non-null    object
 16  AVG_PACE           140 non-null    object
dt

In [9]:
XC_2016_df.head()

,YEAR,DATE,MEET,COURSE,DISTANCE (meters),WEATHER,WEATHER2,DIVISION,PLACE,FIRST_NAME,LAST_NAME,FULL_NAME,YEAR.1,SCHOOL,SECTION,OVERALL TIME,AVG_PACE
0,2016,11/29/2014,CIF State Cross Country,Woodward Park,5000,Sunny,70,4,1,Callum,Bolger,Callum Bolger,SR,San Luis Obispo,SS,14:57.4,04:48.5
1,2016,11/29/2014,CIF State Cross Country,Woodward Park,5000,Sunny,70,4,2,Jared,Rodriguez,Jared Rodriguez,SR,Foothill Technology,SS,15:29.3,04:58.8
2,2016,11/29/2014,CIF State Cross Country,Woodward Park,5000,Sunny,70,4,3,Joseph,Garcia,Joseph Garcia,SR,Salesian,SS,15:33.9,05:00.3
3,2016,11/29/2014,CIF State Cross Country,Woodward Park,5000,Sunny,70,4,4,Ryan,Cutter,Ryan Cutter,SR,Hercules,NCS,15:39.9,05:02.2
4,2016,11/29/2014,CIF State Cross Country,Woodward Park,5000,Sunny,70,4,5,Anthony,Grover,Anthony Grover,FR,JSerra,SS,15:40.1,05:02.3


   
  ## Data Formatting/Analysis

In [21]:
# convert the "DATE" column to a date object
XC_2016_df['DATE'] = pd.to_datetime(XC_2016_df['DATE'])

In [22]:
XC_2016_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   YEAR               170 non-null    int64         
 1   DATE               170 non-null    datetime64[ns]
 2   MEET               170 non-null    object        
 3   COURSE             170 non-null    object        
 4   DISTANCE (meters)  170 non-null    int64         
 5   WEATHER            170 non-null    object        
 6   WEATHER2           170 non-null    int64         
 7   DIVISION           170 non-null    int64         
 8   PLACE              170 non-null    int64         
 9   FIRST_NAME         170 non-null    object        
 10  LAST_NAME          170 non-null    object        
 11  FULL_NAME          170 non-null    object        
 12  YEAR.1             170 non-null    object        
 13  SCHOOL             170 non-null    object        
 14  SECTION   

In [23]:
# check for nulls
check_null(XC_2016_df)

YEAR                  0.000000
DATE                  0.000000
MEET                  0.000000
COURSE                0.000000
DISTANCE (meters)     0.000000
WEATHER               0.000000
WEATHER2              0.000000
DIVISION              0.000000
PLACE                 0.000000
FIRST_NAME            0.000000
LAST_NAME             0.000000
FULL_NAME             0.000000
YEAR.1                0.000000
SCHOOL                0.000000
SECTION               0.000000
OVERALL TIME          0.000000
AVG_PACE             17.647059
dtype: float64

In [26]:
# Load File with College Data added to High School XC Results File

DATA_DIR = "/Users/tinalount/Desktop/Data/"

df_college = pd.read_csv(f"{DATA_DIR}/CIF_XC_2016 HS_to_COLLEGE.csv")

In [27]:
df_college.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   YEAR                      150 non-null    float64
 1   DATE                      150 non-null    object 
 2   MEET                      150 non-null    object 
 3   COURSE                    150 non-null    object 
 4   DISTANCE (meters)         150 non-null    float64
 5   WEATHER                   150 non-null    object 
 6   WEATHER2                  150 non-null    float64
 7   DIVISION                  150 non-null    float64
 8   PLACE                     150 non-null    float64
 9   FIRST_NAME                150 non-null    object 
 10  LAST_NAME                 150 non-null    object 
 11  FULL_NAME                 150 non-null    object 
 12  YEAR.1                    150 non-null    object 
 13  SCHOOL                    150 non-null    object 
 14  SECTION   

In [25]:
# convert the "OVERALL_TIME" column to a time format column with minutes, seconds, and milliseconds data
#XC_2016_df['OVERALL_TIME'] = pd.to_datetime(XC_2016_df['OVERALL_TIME'], format='%M:%S.%f').dt.strftime('%M:%S.%f')
